In [1]:
!pip install -U 'scikit-learn<0.24'
!pip install sklearn-crfsuite

# YOU NEED TO RESTART THE RUNTIME!!!

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.8 MB 7.8 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.23.2 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.23.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 965 kB 8.7 MB/s 


In [3]:
# Run this cell to mount your drive to this notebook in order to read the datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import re
import json
import pandas as pd
import numpy as np
import pickle 
import warnings
warnings.filterwarnings("ignore")

## Read Dataset

In [5]:
# Put the folder path where the datasets are located
PATH = "/content/drive/MyDrive/cs445/hw2/dataset"

In [6]:
# function to read data, return list of tuples each tuple represents a token contains word, pos tag, chunk tag, and ner tag
def read_data(filename) -> list:

  my_list = []
  file1 = open(PATH + filename, 'r')
  count = 0
  find_sentence = []
    
  while True:

      # Get next line from file
      line = file1.readline()

      #end of file is reached
      if not line:
          break

      line=line.split()

      if len(line) != 0:
        if line[0]!= "-DOCSTART-" and line[1]!= "-X-":
          temp = (line[0], line[1] ,line[2], line[3])
          find_sentence.append(temp)
        else: 
          line = file1.readline()

      elif len(line) == 0: #if line is empty; advance to next sentence
        my_list.append(find_sentence)
        find_sentence = []
    
  return my_list

# https://www.geeksforgeeks.org/read-a-file-line-by-line-in-python/

In [7]:
# read data with your custom function
train_data = read_data("/train.txt")
val_data = read_data("/valid.txt")
test_data = read_data("/test.txt")

In [8]:
print(len(train_data), len(val_data),len(test_data))

14041 3250 3453


# Create Gazetteer

In [ ]:
# load wikipedia pages
import json
from os import listdir
from os.path import isfile, join
json_path = "/content/drive/MyDrive/cs445/hw2/wikipedia_pages/"


In [ ]:
# I saved the gazetteer to drive and do not run below part again.

In [ ]:
onlyfiles = [f for f in listdir(json_path) if isfile(join(json_path, f))]
len(onlyfiles)

In [ ]:
# Opening JSON file
import re

my_gazetteer = {}
counter = 0

for k in onlyfiles:
  f = open(json_path + k)

  # returns JSON object as a dictionary
  data = json.load(f)
    
  # find interwiki links from the content of the pages  
  urls = re.findall(r'"&gt;([^\&]+)', data["text"])

  for m in urls:
        if m not in my_gazetteer:
            my_gazetteer[m] = counter
            counter = counter + 1
  
  #print("URL List", urls)
  #print("number of urls found:", len(urls), "check website via: ", data["url"])

In [ ]:
my_gazetteer

In [ ]:
with open('/content/drive/MyDrive/cs445/hw2/saved_dictionary.pkl', 'wb') as f:
    pickle.dump(my_gazetteer, f)

In [ ]:
# Use the saved gazetteer.

In [ ]:
with open('/content/drive/MyDrive/cs445/hw2/saved_dictionary.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

In [ ]:
len(loaded_dict)

In [ ]:
# you can also define rules to improve your gazetteer  
#check if lowercase
for k, v in list(loaded_dict.items()):
    if k[0].islower():
        del loaded_dict[k]

In [ ]:
# print the size of your gazetteer
len(loaded_dict)

In [ ]:
my_gazetteer = loaded_dict

# Models

## Conditional Random Fields (CRF)

### Extract features for CRF

In [9]:
import nltk
from nltk.stem import PorterStemmer

nltk.download('stopwords')
from nltk.corpus import stopwords

import sklearn_crfsuite
from sklearn.metrics import make_scorer
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
# create a function to extract features for each token

ps = PorterStemmer()

def convert_shape(my_str: str):
    x = ""
    for k in my_str:
        if k.isdigit():
            x = x + "d"
        elif k.isalpha():
            if k.isupper():
                x = x + "X"
            else:     
                x = x + "x"
        else: 
            x = x + k
    return x

def convert_short(my_str: str):
    x = ""
    for k in range(len(my_str)):
        if k != len(my_str)-1:
            if my_str[k] != my_str[k+1]:
                x = x + my_str[k] 
            else: 
                pass
    x = x + my_str[k] 
    return x


def token2features(sentence: list, idx: int) -> dict:
    word = sentence[idx][0]
    postag = sentence[idx][1]
    chunk_tag = sentence[idx][2]

    features = {
        'stem': ps.stem(word),
        'postag': postag,
        'chunk_tag': chunk_tag,
        'BOS': False,
        'EOS': False,
        'word_starts_with_upper()': word[0].isupper(),
        'word_shape': convert_shape(word), 
        'short_word_shape':convert_short(convert_shape(word)),  
        'contains_number': any(char.isdigit() for char in word),
        'contains_hyphen': "-" in word,
        'upper_digit_dash': any(char.isdigit() for char in word) and 
        any(char.isupper() for char in word) and "-" in word,
        'contains_prefix': word[:4],
        'contains_suffix': word[-4:],
        'word_isupper()': word.isupper(),       
        'stopwords_contains()': word in stopwords.words('english'),
        'gazetteer_contains()': word in my_gazetteer,

        #additional features:
        'word.lower()': word.lower(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        
    }

    if idx > 0:
        word1 = sentence[idx-1][0]
        postag1 = sentence[idx-1][1]
        chunk_tag1 = sentence[idx-1][2]

        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:chunk_tag1': chunk_tag1,
           
            '-1:neighbors': word1,
            '-1:neighbors_short_shape': convert_short(convert_shape(word1)),
            '-1:neighbors_shape': convert_shape(word1),
        })

    else:
        features['BOS'] = True

    if idx < len(sentence)-1:
        word1 = sentence[idx+1][0]
        postag1 = sentence[idx+1][1]
        chunk_tag1 = sentence[idx+1][2]

        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:chunk_tag1': chunk_tag1,
            
            '+1:neighbors': word1,
            '+1:neighbors_short_shape': convert_short(convert_shape(word1)),
            '+1:neighbors_shape': convert_shape(word1),
        })
    else:
        features['EOS'] = True

    return features

In [11]:
# define function to process each token given a sentence
def sent2features(sentence: list) -> list:
  return [token2features(sentence, i) for i in range(len(sentence))]

# get named entity labels from the sentence
def sent2labels(sentence: list) -> list:
  return [label for token, postag, chunk, label in sentence]

In [ ]:
# prepare inputs and labels
train_sents = [sent2features(s) for s in train_data]
val_sents = [sent2features(s) for s in val_data]
test_sents = [sent2features(s) for s in test_data]

train_labels = [sent2labels(s) for s in train_data]
val_labels = [sent2labels(s) for s in val_data]
test_labels = [sent2labels(s) for s in test_data]

In [ ]:
#train_sents[:2]

In [ ]:
# Set the hyperparameter space that will be scanned.
parameters = {'algorithm':["lbfgs","l2sgd"]} #which algorithm works better

# initialize GridSearchCV for CRF 
crf = sklearn_crfsuite.CRF(max_iterations=100,
    all_possible_transitions=True)

pipe  = GridSearchCV(crf, cv=5,param_grid=parameters)

# fitting the model for grid search 
pipe.fit(train_sents,train_labels,X_dev=val_sents, y_dev=val_labels)

GridSearchCV(cv=5,
             estimator=CRF(all_possible_transitions=True, keep_tempfiles=None,
                           max_iterations=100),
             param_grid={'algorithm': ['lbfgs', 'l2sgd']})

In [ ]:
# print best parameter after tuning 
print(pipe.best_params_)

{'algorithm': 'l2sgd'}


In [ ]:
# initialize and train a crf model with best hyper-parameters
crf = sklearn_crfsuite.CRF(
    algorithm='l2sgd',
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(train_sents,train_labels)

CRF(algorithm='l2sgd', all_possible_transitions=True, keep_tempfiles=None,
    max_iterations=100)

In [ ]:
# calculate f1-score and classification report for test using sklearn_crfsuite.metrics class
labels = list(crf.classes_)
labels.remove('O')
print(labels)

sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)

y_pred = crf.predict(test_sents)

print("F1:", metrics.flat_f1_score(test_labels, y_pred,
                      average='weighted', labels=labels))

print(sklearn_crfsuite.metrics.flat_classification_report(
    test_labels, y_pred, labels=sorted_labels, digits=3
))

['B-ORG', 'B-MISC', 'B-PER', 'I-PER', 'B-LOC', 'I-ORG', 'I-MISC', 'I-LOC']
F1: 0.8364542815134832
              precision    recall  f1-score   support

       B-LOC      0.863     0.880     0.871      1668
       I-LOC      0.773     0.743     0.758       257
      B-MISC      0.845     0.759     0.800       702
      I-MISC      0.676     0.676     0.676       216
       B-ORG      0.815     0.790     0.802      1661
       I-ORG      0.698     0.824     0.756       835
       B-PER      0.859     0.870     0.864      1617
       I-PER      0.898     0.952     0.924      1156

   micro avg      0.829     0.844     0.836      8112
   macro avg      0.803     0.812     0.806      8112
weighted avg      0.831     0.844     0.836      8112



In [ ]:
len(train_sents)

14041

In [ ]:
len(val_sents)

3250

In [ ]:
# start from the stem of the token and add features one by one and train a new model with each feature that you add

#features sorted based on assignment document
list_features= [
'stem',         
'postag',
'chunk_tag',
'BOS',
'EOS',
'word_starts_with_upper()',
'word_shape',
'short_word_shape',
'contains_number',
'contains_hyphen',
'upper_digit_dash',
'contains_prefix',
'contains_suffix',
'word_isupper()',
'stopwords_contains()',
'word.isdigit()',
'word.istitle()',
'word.lower()',

'+1:neighbors',
'-1:neighbors',

'+1:neighbors_shape',
'-1:neighbors_shape',

'+1:neighbors_short_shape',
'-1:neighbors_short_shape',

'+1:postag',
'-1:postag',

'+1:chunk_tag1',
'-1:chunk_tag1',

'+1:word.istitle()',
'-1:word.istitle()',

'+1:word.isupper()',
'-1:word.isupper()',

'+1:word.lower()',
'-1:word.lower()',

'gazetteer_contains()',
] 

crf = sklearn_crfsuite.CRF(
    algorithm='l2sgd',
    max_iterations=100,
    all_possible_transitions=True
)

precision_results = []
recall_results = []
f1_results = []
classification_report_lists = []

for g in range(len(list_features)):
    temp_features = list_features[:g]
    
    c_train_sents = []

    for i in train_sents: #each sentence
        c_train_words = []
        for s in i: #each word
            new_dict = {}
            for k, v in list(s.items()): #features of each word
                if k in temp_features:
                    new_dict[k] = v
            c_train_words.append(new_dict)
        c_train_sents.append(c_train_words)
        
    #should I apply feature subset function to test sents as well?

    crf.fit(c_train_sents,train_labels)

    y_pred = crf.predict(val_sents)

    f1_results.append(metrics.flat_f1_score(val_labels, y_pred,
                        average='weighted', labels=labels))
    
    recall_results.append(metrics.flat_recall_score(val_labels, y_pred,
                        average='weighted', labels=labels))
    
    precision_results.append(metrics.flat_precision_score(val_labels, y_pred,
                        average='weighted', labels=labels))

    classification_report_lists.append(sklearn_crfsuite.metrics.flat_classification_report(
        val_labels, y_pred, labels=sorted_labels, digits=3))

In [ ]:
with open("/content/drive/MyDrive/cs445/hw2/f1_results.pkl'", "wb") as f:   #Pickling
   pickle.dump(f1_results, f)

with open("/content/drive/MyDrive/cs445/hw2/recall_results.pkl'", "wb") as f:   #Pickling
   pickle.dump(recall_results, f)

with open("/content/drive/MyDrive/cs445/hw2/precision_results.pkl'", "wb") as f:   #Pickling
   pickle.dump(precision_results, f)
 
with open("/content/drive/MyDrive/cs445/hw2/classification_report_lists.pkl'", "wb") as f:   #Pickling
   pickle.dump(classification_report_lists, f)
 

In [ ]:
with open("/content/drive/MyDrive/cs445/hw2/f1_results.pkl'", "rb") as f:   #Pickling
   f1_loaded = pickle.load(f)

with open("/content/drive/MyDrive/cs445/hw2/recall_results.pkl'", "rb") as f:   #Pickling
   recall_loaded = pickle.load(f)

with open("/content/drive/MyDrive/cs445/hw2/precision_results.pkl'", "rb") as f:   #Pickling
   precision_loaded = pickle.load(f)

with open("/content/drive/MyDrive/cs445/hw2/classification_report_lists.pkl'", "rb") as f:   #Pickling
   classification_loaded = pickle.load(f)

In [ ]:
len(f1_results)

35

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl

data = {
  "f1": f1_results,
  "recall": recall_results,
  "precision": precision_results
}

df = pd.DataFrame(data, index = [list_features])

print(df) 
# display the result table

                                f1    recall  precision
stem                      0.010779  0.006277   0.056931
postag                    0.684798  0.588516   0.826873
chunk_tag                 0.782985  0.746251   0.831331
BOS                       0.767779  0.752179   0.791860
EOS                       0.772295  0.754272   0.799179
word_starts_with_upper()  0.771120  0.755783   0.796799
word_shape                0.795270  0.795885   0.799104
short_word_shape          0.810482  0.807509   0.816249
contains_number           0.806470  0.805765   0.809727
contains_hyphen           0.803524  0.798791   0.810854
upper_digit_dash          0.802260  0.801581   0.806701
contains_prefix           0.806804  0.804138   0.814102
contains_suffix           0.830438  0.827502   0.835824
word_isupper()            0.850663  0.844938   0.857965
stopwords_contains()      0.847702  0.843543   0.853329
word.isdigit()            0.847628  0.841567   0.855144
word.istitle()            0.848347  0.844822   0

In [ ]:
# display the classification report for the best model
print(classification_loaded[-1])

              precision    recall  f1-score   support

       B-LOC      0.903     0.929     0.916      1837
       I-LOC      0.899     0.868     0.883       257
      B-MISC      0.922     0.854     0.886       922
      I-MISC      0.877     0.723     0.792       346
       B-ORG      0.887     0.831     0.858      1341
       I-ORG      0.789     0.847     0.817       751
       B-PER      0.921     0.905     0.913      1842
       I-PER      0.954     0.954     0.954      1307

   micro avg      0.902     0.887     0.895      8603
   macro avg      0.894     0.864     0.877      8603
weighted avg      0.903     0.887     0.894      8603



In [ ]:
#merge val and train set in order to use all data we have before testing!!!
train_sents.extend(val_sents)
train_labels.extend(val_labels)

In [ ]:
crf.fit(c_train_sents,train_labels)

y_pred = crf.predict(test_sents)

print(metrics.flat_f1_score(test_labels, y_pred,
                    average='weighted', labels=labels))

print(metrics.flat_recall_score(test_labels, y_pred,
                    average='weighted', labels=labels))

print(metrics.flat_precision_score(test_labels, y_pred,
                    average='weighted', labels=labels))

print(sklearn_crfsuite.metrics.flat_classification_report(
    test_labels, y_pred, labels=sorted_labels, digits=3))

0.8323907245627405
0.8348126232741617
0.8317356595515588
              precision    recall  f1-score   support

       B-LOC      0.869     0.882     0.876      1668
       I-LOC      0.828     0.770     0.798       257
      B-MISC      0.810     0.769     0.789       702
      I-MISC      0.580     0.690     0.630       216
       B-ORG      0.835     0.757     0.794      1661
       I-ORG      0.730     0.782     0.755       835
       B-PER      0.845     0.866     0.856      1617
       I-PER      0.889     0.952     0.919      1156

   micro avg      0.831     0.835     0.833      8112
   macro avg      0.798     0.809     0.802      8112
weighted avg      0.832     0.835     0.832      8112



## Recurrent Neural Network (RNN)

In [12]:
import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Model, Input, Sequential
from keras.layers import Dense, Flatten, Embedding, Input, Dropout, LSTM, TimeDistributed, Bidirectional
from keras.callbacks import ModelCheckpoint

from gensim.models import Word2Vec
import gensim.downloader as api
from tensorflow.keras.utils import to_categorical

In [13]:
!pip install seqeval
from seqeval.metrics import f1_score, precision_score, recall_score, classification_report

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.8 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=2863e760a79ed7cd738c2b80d41d1c998f55514b3a69a50ea07af979e78d0407
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [14]:
train_labels = [sent2labels(s) for s in train_data]
val_labels = [sent2labels(s) for s in val_data]
test_labels = [sent2labels(s) for s in test_data]

In [15]:
# find unique labels and create dictionary to map each label to a unique integer value

# function to get unique values
def unique(list1):
  
    unique_dict = {}
    # initialize a null list
      
    # traverse for all elements
    for x in list1: 
        for k in x:
        # check if exists in unique_list or not
            if k not in unique_dict:
                unique_dict[k] = 1
    # print list
    return unique_dict.keys()
      
    
unique(train_labels)

dict_keys(['B-ORG', 'O', 'B-MISC', 'B-PER', 'I-PER', 'B-LOC', 'I-ORG', 'I-MISC', 'I-LOC'])

In [16]:
x = list(unique(train_labels))

In [17]:
n_tag = len(list(unique(train_labels)))

In [18]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(x)

LabelEncoder()

In [19]:
le.classes_

array(['B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG',
       'I-PER', 'O'], dtype='<U6')

In [20]:
le.transform(['O'])

array([8])

In [21]:
train_labels[:3]

[['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O'],
 ['B-PER', 'I-PER'],
 ['B-LOC', 'O']]

In [22]:
before_padding_train_labels = train_labels.copy()
before_padding_val_labels = val_labels.copy()
before_padding_test_labels = test_labels.copy()

In [23]:
for k in range(len(train_labels)):
    train_labels[k] = list(le.transform(train_labels[k]))

for k in range(len(val_labels)):
    val_labels[k] = list(le.transform(val_labels[k]))

for k in range(len(test_labels)):
    test_labels[k] = list(le.transform(test_labels[k]))


In [24]:
train_labels[:3]

[[2, 8, 1, 8, 8, 8, 1, 8, 8], [3, 7], [0, 8]]

In [25]:
def sent2words(sentence: list) -> list:
  return [token for token, postag, chunk, label in sentence]

def sent2postag(sentence: list) -> list:
  return [postag for token, postag, chunk, label in sentence]

In [26]:
train_sents = [sent2words(s) for s in train_data]
val_sents = [sent2words(s) for s in val_data]
test_sents = [sent2words(s) for s in test_data]

p_train_sents = [sent2postag(s) for s in train_data]
p_val_sents = [sent2postag(s) for s in val_data]
p_test_sents = [sent2postag(s) for s in test_data]

In [27]:
# Create your own word embeddings from scratch and load a pretrained word embeddings

# You can check https://radimrehurek.com/gensim/models/word2vec.html for training a word embeddings from scratch
train_wv = Word2Vec(sentences = train_sents, size=20, window=5, min_count=1, workers=4) #multiclass word2vec

# You can check https://radimrehurek.com/gensim/auto_examples/howtos/run_downloader_api.html and https://github.com/RaRe-Technologies/gensim-data for loading pretrained word embeddings. 
api_wv = api.load('glove-twitter-25')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [28]:
train_labels[:3]

[[2, 8, 1, 8, 8, 8, 1, 8, 8], [3, 7], [0, 8]]

In [29]:
train_sents[:3]

[['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'],
 ['Peter', 'Blackburn'],
 ['BRUSSELS', '1996-08-22']]

In [30]:
#find average
max = 0
for i in test_sents:
  if max < len(i):
    max = len(i)
max

124

In [31]:
#train_labels[0]

In [32]:
# preprare your dataset for RNN classifier (you need to add padding to labels as well)

tokenizer = Tokenizer() 
tokenizer.fit_on_texts(train_sents)

train_sents = tokenizer.texts_to_sequences(train_sents)
val_sents = tokenizer.texts_to_sequences(val_sents)
test_sents = tokenizer.texts_to_sequences(test_sents)

vocab_size = len(tokenizer.word_index) + 1 
maxlen = 124
#maxlen = 80

train_sents = pad_sequences(train_sents, padding='post', maxlen=maxlen)
val_sents = pad_sequences(val_sents, padding='post', maxlen=maxlen)
test_sents = pad_sequences(test_sents, padding='post', maxlen=maxlen)

train_labels = pad_sequences(train_labels, padding='post', maxlen=maxlen, value=8)
val_labels = pad_sequences(val_labels, padding='post', maxlen=maxlen, value=8)
test_labels = pad_sequences(test_labels, padding='post', maxlen=maxlen, value=8)

print(train_sents[:5]) 

[[  988 10950   204   628     6  3938   215  5773     2     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0]
 [  773  1871     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0    

In [33]:
train_labels[:3]

array([[2, 8, 1, 8, 8, 8, 1, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8],
       [3, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8],
       [0, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8,

In [34]:
train_labels.shape

(14041, 124)

In [35]:
train_labels = [to_categorical(i, num_classes=n_tag) for i in train_labels]

val_labels = [to_categorical(i, num_classes=n_tag) for i in val_labels]

test_labels = [to_categorical(i, num_classes=n_tag) for i in test_labels]

In [36]:
train_labels[0].shape

(124, 9)

In [37]:
n_tag

9

#RANDOMLY INITILIAZED

In [48]:
from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score
from tensorflow.keras import layers
from keras import optimizers 
from keras.layers.advanced_activations import LeakyReLU

In [39]:
# Create Embedding Matrices and Layers

input_dim = vocab_size
output_dim = 100

def get_bilstm_lstm_model(embedding_class, embedding_matrix): # embedding_class, embedding_matrix
    model = Sequential()

    # Add Embedding layer
    #model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=maxlen))

    if embedding_class == 0:
        model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=maxlen))
    elif embedding_class == 1:
        model.add(Embedding(input_dim, output_dim=20, input_length=maxlen, weights=[embedding_matrix], trainable=True))
    else:
        model.add(Embedding(input_dim, output_dim=25, input_length=maxlen, weights=[embedding_matrix], trainable=True))
    
    # Add bidirectional LSTM
    model.add(Bidirectional(LSTM(128,return_sequences=True)))
    model.add(Dropout(0.4))

    # Add timeDistributed Layer
    model.add(TimeDistributed(Dense(32, activation="relu")))
    model.add(TimeDistributed(Dense(n_tag, activation="softmax")))

    #Optimiser 
    optimizer = tf.keras.optimizers.Adam()
    #optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
    #optimizer='rmsprop'

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    
    return model

In [40]:
# Create your models and train them

def train_model(X_train, y_train, X_val ,y_val, model):
    loss = []
    for i in range(10):
        # fit model for one epoch on this sequence
        hist = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=64, verbose=1, epochs=1)
        loss.append(hist.history['loss'][0])
    return loss, model

In [ ]:
loss, random_model = train_model(train_sents, np.array(train_labels), val_sents ,np.array(val_labels) ,get_bilstm_lstm_model(0,0))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 124, 100)          2101000   
                                                                 
 bidirectional (Bidirectiona  (None, 124, 256)         234496    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 124, 256)          0         
                                                                 
 time_distributed (TimeDistr  (None, 124, 32)          8224      
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 124, 9)           297       
 tributed)                                                       
                                                        

#Word embeddings trained from scratch with gensim

In [53]:
from gensim.models import KeyedVectors

#vocab_len = len(train_wv.wv.vocab) + 1

train_wv.save("train_wv.wordvectors")
embedding_vector = KeyedVectors.load("train_wv.wordvectors", mmap='r')

embedding_matrix = np.zeros((input_dim, 20))
for word, i in tokenizer.word_index.items():
    try:
      #print(embedding_vector[word].shape, word)
      embedding_matrix[i-1] = embedding_vector[word]
    except KeyError:
      #print("key error", i, word)
      pass


In [ ]:
loss, wv_model = train_model(train_sents, np.array(train_labels), val_sents ,np.array(val_labels) ,get_bilstm_lstm_model(1,embedding_matrix))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 124, 20)           420200    
                                                                 
 bidirectional_1 (Bidirectio  (None, 124, 256)         152576    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 124, 256)          0         
                                                                 
 time_distributed_2 (TimeDis  (None, 124, 32)          8224      
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 124, 9)           297       
 tributed)                                                       
                                                      

#Pretrained word embeddings from gensim.api

In [54]:
api_wv.save("api_wv.wordvectors")
embedding_vector = KeyedVectors.load("api_wv.wordvectors", mmap='r')

embedding_matrix2 = np.zeros((input_dim, 25))

unknown_counter = 0
for word, i in tokenizer.word_index.items():
    try:
      #print(embedding_vector[word], word)
      embedding_matrix2[i-1] = embedding_vector[word]
    except KeyError:
      #print("key error", i, word)
      unknown_counter = unknown_counter + 1

#print(unknown_counter) 

In [ ]:
loss , api_model = train_model(train_sents, np.array(train_labels), val_sents ,np.array(val_labels) ,get_bilstm_lstm_model(2,embedding_matrix2))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 124, 25)           525250    
                                                                 
 bidirectional_2 (Bidirectio  (None, 124, 256)         157696    
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 124, 256)          0         
                                                                 
 time_distributed_4 (TimeDis  (None, 124, 32)          8224      
 tributed)                                                       
                                                                 
 time_distributed_5 (TimeDis  (None, 124, 9)           297       
 tributed)                                                       
                                                      

In [41]:
# define a function to remove paddings and align labels and tokens
def align_predictions(predictions:np.array, label_ids:np.array):
  predictions = np.argmax(predictions, axis=-1) 
  label_ids = np.argmax(label_ids, axis=-1) 

  preds_updated = []
  labels_updated = []  

  for k in range(len(predictions)):
    preds_updated.append(list(le.inverse_transform(predictions[k])))

  for k in range(len(label_ids)):
    labels_updated.append(list(le.inverse_transform(label_ids[k])))
  
  preds_padding_removed = []
  for i in range(len(before_padding_test_labels)):
    preds_padding_removed.append(preds_updated[i][:len(before_padding_test_labels[i])])

  return preds_padding_removed, labels_updated 

#RESULTS

In [ ]:
# Evaluate your models with functions of seqeval library

preds = random_model.predict(test_sents)
preds_updated,labels_updated  = align_predictions(preds,np.array(test_labels))

print(classification_report(before_padding_test_labels, preds_updated))
#RANDOMLY INITILIAZED FIRST MODEL

              precision    recall  f1-score   support

         LOC       0.41      0.41      0.41      1668
        MISC       0.38      0.38      0.38       702
         ORG       0.50      0.40      0.44      1661
         PER       0.29      0.16      0.21      1617

   micro avg       0.41      0.33      0.37      5648
   macro avg       0.40      0.34      0.36      5648
weighted avg       0.40      0.33      0.36      5648



In [ ]:
preds = wv_model.predict(test_sents)
preds_updated,labels_updated  = align_predictions(preds,np.array(test_labels))

print(classification_report(before_padding_test_labels, preds_updated))
#WV FIRST MODEL

              precision    recall  f1-score   support

         LOC       0.42      0.40      0.41      1668
        MISC       0.44      0.38      0.41       702
         ORG       0.51      0.45      0.48      1661
         PER       0.30      0.16      0.20      1617

   micro avg       0.43      0.34      0.38      5648
   macro avg       0.42      0.35      0.37      5648
weighted avg       0.41      0.34      0.37      5648



In [ ]:
preds = api_model.predict(test_sents)
preds_updated,labels_updated  = align_predictions(preds,np.array(test_labels))

print(classification_report(before_padding_test_labels, preds_updated))
#API FIRST MODEL

              precision    recall  f1-score   support

         LOC       0.43      0.39      0.41      1668
        MISC       0.36      0.32      0.34       702
         ORG       0.53      0.41      0.46      1661
         PER       0.26      0.16      0.20      1617

   micro avg       0.41      0.32      0.36      5648
   macro avg       0.40      0.32      0.35      5648
weighted avg       0.41      0.32      0.36      5648



In [ ]:

def get_bilstm_lstm_model(embedding_class, embedding_matrix): # embedding_class, embedding_matrix
    model = Sequential()

    if embedding_class == 0:
        model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=maxlen))
    elif embedding_class == 1:
        model.add(Embedding(input_dim, output_dim=20, input_length=maxlen, weights=[embedding_matrix], trainable=True))
    else:
        model.add(Embedding(input_dim, output_dim=25, input_length=maxlen, weights=[embedding_matrix], trainable=True))
    
    #model.add(LSTM(units=128, return_sequences=True, dropout=0.3, recurrent_dropout=0.3))
    #model.add(Dropout(0.4))

    model.add(Bidirectional(LSTM(128,return_sequences=True)))
    model.add(Dropout(0.4))

    # Add timeDistributed Layer
    model.add(TimeDistributed(Dense(16, activation="relu")))
    model.add(TimeDistributed(Dense(n_tag, activation="softmax")))

    #Optimiser 
    optimizer = tf.keras.optimizers.Adam()

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    
    return model

In [ ]:
loss, random_model = train_model(train_sents, np.array(train_labels), val_sents ,np.array(val_labels) ,get_bilstm_lstm_model(0,0))

preds = random_model.predict(test_sents)
preds_updated,labels_updated  = align_predictions(preds,np.array(test_labels))

print(classification_report(before_padding_test_labels, preds_updated))
#RANDOMLY INITILIAZED SECOND MODEL

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 124, 100)          2101000   
                                                                 
 bidirectional_3 (Bidirectio  (None, 124, 256)         234496    
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 124, 256)          0         
                                                                 
 time_distributed_6 (TimeDis  (None, 124, 16)          4112      
 tributed)                                                       
                                                                 
 time_distributed_7 (TimeDis  (None, 124, 9)           153       
 tributed)                                                       
                                                      

In [ ]:
loss, wv_model = train_model(train_sents, np.array(train_labels), val_sents ,np.array(val_labels) ,get_bilstm_lstm_model(1,embedding_matrix))

preds = wv_model.predict(test_sents)
preds_updated,labels_updated  = align_predictions(preds,np.array(test_labels))

print(classification_report(before_padding_test_labels, preds_updated))
#WV SECOND MODEL

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 124, 20)           420200    
                                                                 
 bidirectional_4 (Bidirectio  (None, 124, 256)         152576    
 nal)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 124, 256)          0         
                                                                 
 time_distributed_8 (TimeDis  (None, 124, 16)          4112      
 tributed)                                                       
                                                                 
 time_distributed_9 (TimeDis  (None, 124, 9)           153       
 tributed)                                                       
                                                      

In [ ]:
loss, api_model = train_model(train_sents, np.array(train_labels), val_sents ,np.array(val_labels) ,get_bilstm_lstm_model(2,embedding_matrix2))

preds = api_model.predict(test_sents)
preds_updated,labels_updated  = align_predictions(preds,np.array(test_labels))

print(classification_report(before_padding_test_labels, preds_updated))
#API SECOND MODEL

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 124, 25)           525250    
                                                                 
 bidirectional_5 (Bidirectio  (None, 124, 256)         157696    
 nal)                                                            
                                                                 
 dropout_5 (Dropout)         (None, 124, 256)          0         
                                                                 
 time_distributed_10 (TimeDi  (None, 124, 16)          4112      
 stributed)                                                      
                                                                 
 time_distributed_11 (TimeDi  (None, 124, 9)           153       
 stributed)                                                      
                                                      

In [ ]:
def get_bilstm_lstm_model(embedding_class, embedding_matrix): # embedding_class, embedding_matrix
    model = Sequential()

    if embedding_class == 0:
        model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=maxlen))
    elif embedding_class == 1:
        model.add(Embedding(input_dim, output_dim=20, input_length=maxlen, weights=[embedding_matrix], trainable=True))
    else:
        model.add(Embedding(input_dim, output_dim=25, input_length=maxlen, weights=[embedding_matrix], trainable=True))
    
    model.add(Bidirectional(LSTM(64,return_sequences=True)))
    model.add(Dropout(0.4))

    # Add timeDistributed Layer
    model.add(TimeDistributed(Dense(16, activation="relu")))
    model.add(TimeDistributed(Dense(n_tag, activation="softmax")))

    #Optimiser 
    optimizer = tf.keras.optimizers.Adam()

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    
    return model

In [ ]:
loss, random_model = train_model(train_sents, np.array(train_labels), val_sents ,np.array(val_labels) ,get_bilstm_lstm_model(0,0))

preds = random_model.predict(test_sents)
preds_updated,labels_updated  = align_predictions(preds,np.array(test_labels))

print(classification_report(before_padding_test_labels, preds_updated))
#random_model 3 MODEL

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 124, 100)          2101000   
                                                                 
 bidirectional_6 (Bidirectio  (None, 124, 128)         84480     
 nal)                                                            
                                                                 
 dropout_6 (Dropout)         (None, 124, 128)          0         
                                                                 
 time_distributed_12 (TimeDi  (None, 124, 16)          2064      
 stributed)                                                      
                                                                 
 time_distributed_13 (TimeDi  (None, 124, 9)           153       
 stributed)                                                      
                                                      

In [ ]:
loss, wv_model = train_model(train_sents, np.array(train_labels), val_sents ,np.array(val_labels) ,get_bilstm_lstm_model(1,embedding_matrix))

preds = wv_model.predict(test_sents)
preds_updated,labels_updated  = align_predictions(preds,np.array(test_labels))

print(classification_report(before_padding_test_labels, preds_updated))
#wv_model 3 MODEL

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 124, 20)           420200    
                                                                 
 bidirectional_7 (Bidirectio  (None, 124, 128)         43520     
 nal)                                                            
                                                                 
 dropout_7 (Dropout)         (None, 124, 128)          0         
                                                                 
 time_distributed_14 (TimeDi  (None, 124, 16)          2064      
 stributed)                                                      
                                                                 
 time_distributed_15 (TimeDi  (None, 124, 9)           153       
 stributed)                                                      
                                                      

In [ ]:
loss, api_model = train_model(train_sents, np.array(train_labels), val_sents ,np.array(val_labels) ,get_bilstm_lstm_model(2,embedding_matrix2))

preds = api_model.predict(test_sents)
preds_updated,labels_updated  = align_predictions(preds,np.array(test_labels))

print(classification_report(before_padding_test_labels, preds_updated))
#api_model 3 MODEL

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 124, 25)           525250    
                                                                 
 bidirectional_8 (Bidirectio  (None, 124, 128)         46080     
 nal)                                                            
                                                                 
 dropout_8 (Dropout)         (None, 124, 128)          0         
                                                                 
 time_distributed_16 (TimeDi  (None, 124, 16)          2064      
 stributed)                                                      
                                                                 
 time_distributed_17 (TimeDi  (None, 124, 9)           153       
 stributed)                                                      
                                                      

In [ ]:
def get_bilstm_lstm_model(embedding_class, embedding_matrix): # embedding_class, embedding_matrix
    model = Sequential()

    if embedding_class == 0:
        model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=maxlen))
    elif embedding_class == 1:
        model.add(Embedding(input_dim, output_dim=20, input_length=maxlen, weights=[embedding_matrix], trainable=True))
    else:
        model.add(Embedding(input_dim, output_dim=25, input_length=maxlen, weights=[embedding_matrix], trainable=True))
    
    #model.add(LSTM(units=128, return_sequences=True, dropout=0.3, recurrent_dropout=0.3))
    #model.add(Dropout(0.4))

    model.add(Bidirectional(LSTM(64,return_sequences=True)))
    model.add(Dropout(0.4))

    # Add timeDistributed Layer
    model.add(TimeDistributed(Dense(32, activation="relu")))
    model.add(TimeDistributed(Dense(n_tag, activation="softmax")))

    #Optimiser 
    optimizer = tf.keras.optimizers.Adam()

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    
    return model

In [ ]:
loss, random_model = train_model(train_sents, np.array(train_labels), val_sents ,np.array(val_labels) ,get_bilstm_lstm_model(0,0))

preds = random_model.predict(test_sents)
preds_updated,labels_updated  = align_predictions(preds,np.array(test_labels))

print(classification_report(before_padding_test_labels, preds_updated))
#random_model 4 MODEL

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 124, 100)          2101000   
                                                                 
 bidirectional_9 (Bidirectio  (None, 124, 128)         84480     
 nal)                                                            
                                                                 
 dropout_9 (Dropout)         (None, 124, 128)          0         
                                                                 
 time_distributed_18 (TimeDi  (None, 124, 32)          4128      
 stributed)                                                      
                                                                 
 time_distributed_19 (TimeDi  (None, 124, 9)           297       
 stributed)                                                      
                                                      

In [ ]:
loss, wv_model = train_model(train_sents, np.array(train_labels), val_sents ,np.array(val_labels) ,get_bilstm_lstm_model(1,embedding_matrix))

preds = wv_model.predict(test_sents)
preds_updated,labels_updated  = align_predictions(preds,np.array(test_labels))

print(classification_report(before_padding_test_labels, preds_updated))
#wv_model 4 MODEL

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 124, 20)           420200    
                                                                 
 bidirectional_10 (Bidirecti  (None, 124, 128)         43520     
 onal)                                                           
                                                                 
 dropout_10 (Dropout)        (None, 124, 128)          0         
                                                                 
 time_distributed_20 (TimeDi  (None, 124, 32)          4128      
 stributed)                                                      
                                                                 
 time_distributed_21 (TimeDi  (None, 124, 9)           297       
 stributed)                                                      
                                                     

In [ ]:
loss, api_model = train_model(train_sents, np.array(train_labels), val_sents ,np.array(val_labels) ,get_bilstm_lstm_model(2,embedding_matrix2))

preds = api_model.predict(test_sents)
preds_updated,labels_updated  = align_predictions(preds,np.array(test_labels))

print(classification_report(before_padding_test_labels, preds_updated))
#api_model 4 MODEL

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 124, 25)           525250    
                                                                 
 bidirectional_11 (Bidirecti  (None, 124, 128)         46080     
 onal)                                                           
                                                                 
 dropout_11 (Dropout)        (None, 124, 128)          0         
                                                                 
 time_distributed_22 (TimeDi  (None, 124, 32)          4128      
 stributed)                                                      
                                                                 
 time_distributed_23 (TimeDi  (None, 124, 9)           297       
 stributed)                                                      
                                                     

In [ ]:
def get_bilstm_lstm_model(embedding_class, embedding_matrix): # embedding_class, embedding_matrix
    model = Sequential()

    if embedding_class == 0:
        model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=maxlen))
    elif embedding_class == 1:
        model.add(Embedding(input_dim, output_dim=20, input_length=maxlen, weights=[embedding_matrix], trainable=True))
    else:
        model.add(Embedding(input_dim, output_dim=25, input_length=maxlen, weights=[embedding_matrix], trainable=True))
    
    model.add(layers.SimpleRNN(128,return_sequences=True))
    model.add(Dropout(0.4))
    #model.add(Dense(32, activation='relu'))
    model.add(Dense(n_tag, activation='softmax'))

    #model.add(TimeDistributed(Dense(32, activation="relu")))
    #model.add(TimeDistributed(Dense(n_tag, activation="softmax")))

    #Optimiser 
    optimizer = tf.keras.optimizers.Adam()

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    
    return model

In [ ]:
loss, random_model = train_model(train_sents, np.array(train_labels), val_sents ,np.array(val_labels) ,get_bilstm_lstm_model(0,0))

preds = random_model.predict(test_sents)
preds_updated,labels_updated  = align_predictions(preds,np.array(test_labels))

print(classification_report(before_padding_test_labels, preds_updated))
#random_model 5 MODEL

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_17 (Embedding)    (None, 124, 100)          2101000   
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 124, 128)          29312     
                                                                 
 dense_28 (Dense)            (None, 124, 9)            1161      
                                                                 
Total params: 2,131,473
Trainable params: 2,131,473
Non-trainable params: 0
_________________________________________________________________
220/220 [==============================] - 26s 119ms/step - loss: 0.0022 - accuracy: 0.9994 - val_loss: 0.1450 - val_accuracy: 0.9853
              precision    recall  f1-score   support

         LOC       0.40      0.41      0.40      1668
        MISC       0.39      0.35      0.37       702
         O

In [ ]:
loss, wv_model = train_model(train_sents, np.array(train_labels), val_sents ,np.array(val_labels) ,get_bilstm_lstm_model(1,embedding_matrix))

preds = wv_model.predict(test_sents)
preds_updated,labels_updated  = align_predictions(preds,np.array(test_labels))

print(classification_report(before_padding_test_labels, preds_updated))
#wv_model 5 MODEL

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_18 (Embedding)    (None, 124, 20)           420200    
                                                                 
 simple_rnn_6 (SimpleRNN)    (None, 124, 128)          19072     
                                                                 
 dense_29 (Dense)            (None, 124, 9)            1161      
                                                                 
Total params: 440,433
Trainable params: 440,433
Non-trainable params: 0
_________________________________________________________________
220/220 [==============================] - 25s 112ms/step - loss: 0.0060 - accuracy: 0.9984 - val_loss: 0.1248 - val_accuracy: 0.9849
              precision    recall  f1-score   support

         LOC       0.41      0.41      0.41      1668
        MISC       0.37      0.37      0.37       702
         ORG  

In [ ]:
loss, api_model = train_model(train_sents, np.array(train_labels), val_sents ,np.array(val_labels) ,get_bilstm_lstm_model(2,embedding_matrix2))

preds = api_model.predict(test_sents)
preds_updated,labels_updated  = align_predictions(preds,np.array(test_labels))

print(classification_report(before_padding_test_labels, preds_updated))
#API 5 MODEL

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_19 (Embedding)    (None, 124, 25)           525250    
                                                                 
 simple_rnn_7 (SimpleRNN)    (None, 124, 128)          19712     
                                                                 
 dense_30 (Dense)            (None, 124, 9)            1161      
                                                                 
Total params: 546,123
Trainable params: 546,123
Non-trainable params: 0
_________________________________________________________________
220/220 [==============================] - 25s 113ms/step - loss: 0.0127 - accuracy: 0.9964 - val_loss: 0.1177 - val_accuracy: 0.9844
              precision    recall  f1-score   support

         LOC       0.41      0.40      0.41      1668
        MISC       0.35      0.31      0.33       702
         ORG  

In [ ]:
def get_bilstm_lstm_model(embedding_class, embedding_matrix): # embedding_class, embedding_matrix
    model = Sequential()

    if embedding_class == 0:
        model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=maxlen))
    elif embedding_class == 1:
        model.add(Embedding(input_dim, output_dim=20, input_length=maxlen, weights=[embedding_matrix], trainable=True))
    else:
        model.add(Embedding(input_dim, output_dim=25, input_length=maxlen, weights=[embedding_matrix], trainable=True))
    
    model.add(layers.SimpleRNN(128,return_sequences=True))
    model.add(Dropout(0.4))
    
    model.add(Dense(16, activation='relu'))
    model.add(Dense(n_tag, activation='softmax'))

    #Optimiser 
    optimizer = tf.keras.optimizers.Adam()

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    
    return model

In [ ]:
loss, random_model = train_model(train_sents, np.array(train_labels), val_sents ,np.array(val_labels) ,get_bilstm_lstm_model(0,0))

preds = random_model.predict(test_sents)
preds_updated,labels_updated  = align_predictions(preds,np.array(test_labels))

print(classification_report(before_padding_test_labels, preds_updated))
#random_model 6 MODEL

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_20 (Embedding)    (None, 124, 100)          2101000   
                                                                 
 simple_rnn_8 (SimpleRNN)    (None, 124, 128)          29312     
                                                                 
 dense_31 (Dense)            (None, 124, 16)           2064      
                                                                 
 dense_32 (Dense)            (None, 124, 9)            153       
                                                                 
Total params: 2,132,529
Trainable params: 2,132,529
Non-trainable params: 0
_________________________________________________________________
220/220 [==============================] - 26s 120ms/step - loss: 0.0020 - accuracy: 0.9994 - val_loss: 0.1658 - val_accuracy: 0.9849
              precision    recall  f1-sco

In [ ]:
loss, wv_model = train_model(train_sents, np.array(train_labels), val_sents ,np.array(val_labels) ,get_bilstm_lstm_model(1,embedding_matrix))

preds = wv_model.predict(test_sents)
preds_updated,labels_updated  = align_predictions(preds,np.array(test_labels))

print(classification_report(before_padding_test_labels, preds_updated))
#wv_model 6 MODEL

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_21 (Embedding)    (None, 124, 20)           420200    
                                                                 
 simple_rnn_9 (SimpleRNN)    (None, 124, 128)          19072     
                                                                 
 dense_33 (Dense)            (None, 124, 16)           2064      
                                                                 
 dense_34 (Dense)            (None, 124, 9)            153       
                                                                 
Total params: 441,489
Trainable params: 441,489
Non-trainable params: 0
_________________________________________________________________
220/220 [==============================] - 25s 113ms/step - loss: 0.0058 - accuracy: 0.9983 - val_loss: 0.1755 - val_accuracy: 0.9849
              precision    recall  f1-score  

In [ ]:
loss, api_model = train_model(train_sents, np.array(train_labels), val_sents ,np.array(val_labels) ,get_bilstm_lstm_model(2,embedding_matrix2))

preds = api_model.predict(test_sents)
preds_updated,labels_updated  = align_predictions(preds,np.array(test_labels))

print(classification_report(before_padding_test_labels, preds_updated))
#API 6 MODEL

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_22 (Embedding)    (None, 124, 25)           525250    
                                                                 
 simple_rnn_10 (SimpleRNN)   (None, 124, 128)          19712     
                                                                 
 dense_35 (Dense)            (None, 124, 16)           2064      
                                                                 
 dense_36 (Dense)            (None, 124, 9)            153       
                                                                 
Total params: 547,179
Trainable params: 547,179
Non-trainable params: 0
_________________________________________________________________
220/220 [==============================] - 25s 114ms/step - loss: 0.0117 - accuracy: 0.9966 - val_loss: 0.1451 - val_accuracy: 0.9847
              precision    recall  f1-score  

In [43]:
def get_bilstm_lstm_model(embedding_class, embedding_matrix): # embedding_class, embedding_matrix
    model = Sequential()

    if embedding_class == 0:
        model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=maxlen))
    elif embedding_class == 1:
        model.add(Embedding(input_dim, output_dim=20, input_length=maxlen, weights=[embedding_matrix], trainable=True))
    else:
        model.add(Embedding(input_dim, output_dim=25, input_length=maxlen, weights=[embedding_matrix], trainable=True))
    
    model.add(layers.SimpleRNN(64,return_sequences=True))
    model.add(Dropout(0.4))

    model.add(Dense(16, activation='relu'))
    model.add(Dense(n_tag, activation='softmax'))

    #Optimiser 
    optimizer = tf.keras.optimizers.Adam()

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    
    return model

In [ ]:
loss, random_model = train_model(train_sents, np.array(train_labels), val_sents ,np.array(val_labels) ,get_bilstm_lstm_model(0,0))

preds = random_model.predict(test_sents)
preds_updated,labels_updated  = align_predictions(preds,np.array(test_labels))

print(classification_report(before_padding_test_labels, preds_updated))
#random_model 7 MODEL

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_23 (Embedding)    (None, 124, 100)          2101000   
                                                                 
 simple_rnn_11 (SimpleRNN)   (None, 124, 128)          29312     
                                                                 
 dense_37 (Dense)            (None, 124, 16)           2064      
                                                                 
 dense_38 (Dense)            (None, 124, 9)            153       
                                                                 
Total params: 2,132,529
Trainable params: 2,132,529
Non-trainable params: 0
_________________________________________________________________
220/220 [==============================] - 25s 115ms/step - loss: 0.0017 - accuracy: 0.9995 - val_loss: 0.2083 - val_accuracy: 0.9849
              precision    recall  f1-sco

In [ ]:
loss, wv_model = train_model(train_sents, np.array(train_labels), val_sents ,np.array(val_labels) ,get_bilstm_lstm_model(1,embedding_matrix))

preds = wv_model.predict(test_sents)
preds_updated,labels_updated  = align_predictions(preds,np.array(test_labels))

print(classification_report(before_padding_test_labels, preds_updated))
#wv_model 7 MODEL

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_24 (Embedding)    (None, 124, 20)           420200    
                                                                 
 simple_rnn_12 (SimpleRNN)   (None, 124, 128)          19072     
                                                                 
 dense_39 (Dense)            (None, 124, 16)           2064      
                                                                 
 dense_40 (Dense)            (None, 124, 9)            153       
                                                                 
Total params: 441,489
Trainable params: 441,489
Non-trainable params: 0
_________________________________________________________________
220/220 [==============================] - 25s 112ms/step - loss: 0.0057 - accuracy: 0.9984 - val_loss: 0.1687 - val_accuracy: 0.9852
              precision    recall  f1-score  

In [ ]:
loss, api_model = train_model(train_sents, np.array(train_labels), val_sents ,np.array(val_labels) ,get_bilstm_lstm_model(2,embedding_matrix2))

preds = api_model.predict(test_sents)
preds_updated,labels_updated  = align_predictions(preds,np.array(test_labels))

print(classification_report(before_padding_test_labels, preds_updated))
#API 7 MODEL

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_25 (Embedding)    (None, 124, 25)           525250    
                                                                 
 simple_rnn_13 (SimpleRNN)   (None, 124, 128)          19712     
                                                                 
 dense_41 (Dense)            (None, 124, 16)           2064      
                                                                 
 dense_42 (Dense)            (None, 124, 9)            153       
                                                                 
Total params: 547,179
Trainable params: 547,179
Non-trainable params: 0
_________________________________________________________________
220/220 [==============================] - 25s 112ms/step - loss: 0.0118 - accuracy: 0.9967 - val_loss: 0.1344 - val_accuracy: 0.9848
              precision    recall  f1-score  

In [57]:
def get_bilstm_lstm_model(embedding_class, embedding_matrix): # embedding_class, embedding_matrix
    model = Sequential()

    if embedding_class == 0:
        model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=maxlen))
    elif embedding_class == 1:
        model.add(Embedding(input_dim, output_dim=20, input_length=maxlen, weights=[embedding_matrix], trainable=True))
    else:
        model.add(Embedding(input_dim, output_dim=25, input_length=maxlen, weights=[embedding_matrix], trainable=True))
    
    model.add(layers.SimpleRNN(64,return_sequences=True))
    model.add(Dropout(0.4))
    
    model.add(Dense(32, activation='relu'))
    model.add(Dense(n_tag, activation='softmax'))

    #Optimiser 
    optimizer = tf.keras.optimizers.Adam()

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    
    return model

In [58]:
loss, random_model = train_model(train_sents, np.array(train_labels), val_sents ,np.array(val_labels) ,get_bilstm_lstm_model(0,0))

preds = random_model.predict(test_sents)
preds_updated,labels_updated  = align_predictions(preds,np.array(test_labels))

print(classification_report(before_padding_test_labels, preds_updated))

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 124, 100)          2101000   
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 124, 64)           10560     
                                                                 
 dropout (Dropout)           (None, 124, 64)           0         
                                                                 
 dense_6 (Dense)             (None, 124, 32)           2080      
                                                                 
 dense_7 (Dense)             (None, 124, 9)            297       
                                                                 
Total params: 2,113,937
Trainable params: 2,113,937
Non-trainable params: 0
_________________________________________________________________
220/220 [==============================] - 2

In [59]:
loss, random_model = train_model(train_sents, np.array(train_labels), val_sents ,np.array(val_labels) ,get_bilstm_lstm_model(1,embedding_matrix))

preds = random_model.predict(test_sents)
preds_updated,labels_updated  = align_predictions(preds,np.array(test_labels))

print(classification_report(before_padding_test_labels, preds_updated))

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 124, 20)           420200    
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 124, 64)           5440      
                                                                 
 dropout_1 (Dropout)         (None, 124, 64)           0         
                                                                 
 dense_8 (Dense)             (None, 124, 32)           2080      
                                                                 
 dense_9 (Dense)             (None, 124, 9)            297       
                                                                 
Total params: 428,017
Trainable params: 428,017
Non-trainable params: 0
_________________________________________________________________
220/220 [==============================] - 24s 1

In [60]:
loss, random_model = train_model(train_sents, np.array(train_labels), val_sents ,np.array(val_labels) ,get_bilstm_lstm_model(2,embedding_matrix2))

preds = random_model.predict(test_sents)
preds_updated,labels_updated  = align_predictions(preds,np.array(test_labels))

print(classification_report(before_padding_test_labels, preds_updated))

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 124, 25)           525250    
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 124, 64)           5760      
                                                                 
 dropout_2 (Dropout)         (None, 124, 64)           0         
                                                                 
 dense_10 (Dense)            (None, 124, 32)           2080      
                                                                 
 dense_11 (Dense)            (None, 124, 9)            297       
                                                                 
Total params: 533,387
Trainable params: 533,387
Non-trainable params: 0
_________________________________________________________________
220/220 [==============================] - 25s 1

In [72]:
x = np.concatenate((train_sents,val_sents),axis=0)
y = np.concatenate((train_labels,val_labels),axis=0)

In [76]:
input_dim = vocab_size
output_dim = 100

def get_bilstm_lstm_model(embedding_class, embedding_matrix): # embedding_class, embedding_matrix
    model = Sequential()

    # Add Embedding layer
    #model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=maxlen))

    if embedding_class == 0:
        model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=maxlen))
    elif embedding_class == 1:
        model.add(Embedding(input_dim, output_dim=20, input_length=maxlen, weights=[embedding_matrix], trainable=True))
    else:
        model.add(Embedding(input_dim, output_dim=25, input_length=maxlen, weights=[embedding_matrix], trainable=True))
    
    # Add bidirectional LSTM
    model.add(Bidirectional(LSTM(512,return_sequences=True)))
    model.add(Dropout(0.5))

    # Add timeDistributed Layer
    model.add(TimeDistributed(Dense(32, activation="relu")))
    model.add(TimeDistributed(Dense(n_tag, activation="softmax")))

    #Optimiser 
    optimizer = tf.keras.optimizers.Adam()
    #optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
    #optimizer='rmsprop'

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    
    return model

In [77]:
def train_model(X_train, y_train, model):
    loss = []
    for i in range(20):
        # fit model for one epoch on this sequence
        hist = model.fit(X_train, y_train, batch_size=64, verbose=1, epochs=1)
        loss.append(hist.history['loss'][0])
    return loss, model

In [78]:
loss, best_model = train_model(x, np.array(y), get_bilstm_lstm_model(0,0))

preds = best_model.predict(test_sents)
preds_updated,labels_updated  = align_predictions(preds,np.array(test_labels))

print(classification_report(before_padding_test_labels, preds_updated))

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 124, 100)          2101000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 124, 1024)        2510848   
 nal)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 124, 1024)         0         
                                                                 
 time_distributed_2 (TimeDis  (None, 124, 32)          32800     
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 124, 9)           297       
 tributed)                                                       
                                                      

## My Report

# DATASET:
Our dataset is a tuple of 4 features (token, postag, chunk tag, ner label).

We create a few functions such as sent2words, sent2tokens, sent2features. So that, we can use them in order to prepare dataset for the crf model. 

# Create gazetteer:
In order to use for our CRF model, we create gazetteer. We have wikipedia dataset. First, we read each wikipedia file one by one using json. Then, apply some regex functions in order to extract the parts that could be counted as named entities. Then, I applied a rule such as removing words that do not start with capital letter. Using all these words, I created a dictionary. The reason I implemented dictionary is it is efficient in terms of time complexity. Once we train crf model, we benefit using dict.   


# CRF: 
 
*Extracting features:* This is one of the most important parts. Here we extract several features from the dataset. I have 35 features including neighbors features. First a few features definitely affected the performance of model more compared to features that we used later. Despite the improvement rate decreases after a few features, model became much more robust once we add each feature.    


After creating the features, I applied grid search so that I can decide which hyper-parameters work best for the model. I concluded that best params are:


```
CRF(algorithm='l2sgd', all_possible_transitions=True, keep_tempfiles=None,
    max_iterations=100)
```

Then, since it is asked: I observed how each feature changes performance of the model by adding each feature one by one. Results show that each feature affected positively and I sticked with the model contains all features. Then, I **combined my validation and train dataset** so that we can use all the data we have in order to predict test data. We obtained 0.89 F1 score in the validation set and 0.83 in the test set. Here are the results:


```
VALIDATION:

         precision    recall  f1-score   support

       B-LOC      0.903     0.929     0.916      1837
       I-LOC      0.899     0.868     0.883       257
      B-MISC      0.922     0.854     0.886       922
      I-MISC      0.877     0.723     0.792       346
       B-ORG      0.887     0.831     0.858      1341
       I-ORG      0.789     0.847     0.817       751
       B-PER      0.921     0.905     0.913      1842
       I-PER      0.954     0.954     0.954      1307

   micro avg      0.902     0.887     0.895      8603
   macro avg      0.894     0.864     0.877      8603
weighted avg      0.903     0.887     0.894      8603


TEST:
              precision    recall  f1-score   support

       B-LOC      0.869     0.882     0.876      1668
       I-LOC      0.828     0.770     0.798       257
      B-MISC      0.810     0.769     0.789       702
      I-MISC      0.580     0.690     0.630       216
       B-ORG      0.835     0.757     0.794      1661
       I-ORG      0.730     0.782     0.755       835
       B-PER      0.845     0.866     0.856      1617
       I-PER      0.889     0.952     0.919      1156

   micro avg      0.831     0.835     0.833      8112
   macro avg      0.798     0.809     0.802      8112
weighted avg      0.832     0.835     0.832      8112

```


# RNN: 

(similar to previous project's cnn part, hence I used some parts of my previous report and codes.)

In the RNN part, we have tried 3 different embedding strategies: Randomly initialized word embeddings, Word embeddings trained from scratch with gensim, Pretrained word embeddings from gensim.api. In order to do so:
For the Randomly initialized word embeddings, we set embedding dimension, vocabulary size and input length parameters in the embedding layer of our network. So that, weights in the embedding matrix will be initialized randomly.
For the Word embeddings trained from scratch with gensim, we import our trained word2vec model using keyedvectors feature of gensim.models and form the embedding vector. Using the embedding vector, we create the embedding matrix which will be given as a parameter in the embedding layer of our network.
For the Pretrained word embeddings from gensim.api, we import our pre-trained api model using keyedvectors feature of gensim.models and form the embedding vector: Using the embedding vector, we create the embedding matrix which will be given as a parameter in the embedding layer of our network.

We prepared our dataset for RNN model by fitting our tokenizer to the train dataset. Then, we transform each text in texts to a sequence of integers using texts_to_sequences method. We set a maxlen parameter after finding out the maximum length of our dataset and we apply padding.

After preparing the dataset, we continue by building our model:
For the output layer, we set our activation function as softmax since our task is multiclass classification and we want our output between [0,8]. Then, we set loss function categorical_crossentropy for this task. We use adam optimizer since it is the best one for this task after trying other optimizers such as rmsprop.  

We have built 8 different models for each (3) embedding strategy -> in total 24 models for the NER task. 

Trying dense layer with different neurons: we observe that using higher neurons in the layer increased the f1 score around 0.1.

Replacing bilstm layer with simple RNN layer: we observe that bilstm is trained much faster compared to RNN layer. Bilstm layer outperformed the simple RNN layer overall.     

Using randomly initialized embedding layer, we obtained around 38 percent f1 score in the test dataset. 

Using word embeddings trained from scratch with gensim, we obtained around 36 percent accuracy in the test set which is slightly better than randomly initiliazed word embedding results. 

Last but not least, we obtained around 33 percent accuracy in the test dataset using pretrained word embeddings from gensim.api. 

At the end, I trained the model with best parameters (bilstm - 64, dense - 32 ) and merged validation dataset with train dataset. Then, I received 0.40 F1 score. I wanted to try higher neurons for the lstm whether it could work better and give a try for 512 neurons in the bilstm layer and trained for 20 epochs. I was not able to train each model for 20 epochs since it would take so much time. So, I did the longest training for the last model. I obtained 0.43 F1 score which is significantly higher than my other models. 

Overall, we observed that CRF Model outperforms the results of RNN model significantly.

References:

https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html
https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54
